In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import spatialpy
from spatialpy import Model, Species, Parameter, Reaction,\
                      Domain, PlaceInitialCondition, \
                      UniformInitialCondition, \
                      ScatterInitialCondition

# 3D_cylinder(1)

In [7]:
class M3Dcylinder1(Model):
    MIDDLE = 'Middle'
    EDGE1 = 'Edge1'
    EDGE2 = 'Edge2'

    def __init__(self):
        Model.__init__(self, name="3D_cylinder(1)")

        # Domain
        domain = Domain.read_stochss_domain('3D_cylinder.smdl')
        self.add_domain(domain)

        self.staticDomain = True

        # Variables
        A = Species(name="A", diffusion_coefficient=0.1, restrict_to=[self.MIDDLE, self.EDGE1])
        B = Species(name="B", diffusion_coefficient=0.1, restrict_to=[self.MIDDLE, self.EDGE2])
        self.add_species([A, B])

        # Parameters
        k_react = Parameter(name="k_react", expression="1")
        k_creat1 = Parameter(name="k_creat1", expression="100")
        k_creat2 = Parameter(name="k_creat2", expression="100")
        left = Parameter(name="left", expression="0.5092013833059308")
        right = Parameter(name="right", expression="0.505804729089437")
        self.add_parameter([k_react, k_creat1, k_creat2, left, right])

        # Reactions
        r1 = Reaction(name="r1", reactants={}, products={'A': 1}, propensity_function="k_creat1 / left * vol", ode_propensity_function="k_creat1", restrict_to=[self.EDGE1])
        r2 = Reaction(name="r2", reactants={}, products={'B': 1}, propensity_function="k_creat2 / right * vol", ode_propensity_function="k_creat2", restrict_to=[self.EDGE2])
        r3 = Reaction(name="r3", reactants={'A': 1, 'B': 1}, products={}, rate="k_react")
        self.add_reaction([r1, r2, r3])

        # Timespan
        self.timespan(np.arange(0, 500.001, 1), timestep_size=1)

In [8]:
model = M3Dcylinder1()

# Simulation Parameters

In [9]:
def configure_simulation():
    kwargs = {
        # "number_of_trajectories":1,
        # "seed":None,
    }
    return kwargs

In [10]:
kwargs = configure_simulation()
results = model.run(**kwargs)

# Visualization

In [ ]:
results.plot_species('A', animated=True, width=None, height=None)